In [8]:
import os
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm  # Fortschrittsanzeige in Jupyter
from concurrent.futures import ProcessPoolExecutor, as_completed


In [1]:
def process_image(filename, input_folder, output_folder, mapping):
    """
    Lädt ein Labelbild im Palette-Modus ("P"), wendet das Mapping an und speichert das Ergebnis,
    wobei die ursprüngliche Palette (falls vorhanden) übernommen wird.
    """
    try:
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        # Bild im Modus "P" laden – so bleiben die Indexwerte erhalten
        img = Image.open(input_path)
        if img.mode != "P":
            img = img.convert("P")
            
        # Erhalte die Originalwerte als Numpy-Array
        arr = np.array(img)
        
        # Wende das Mapping vektorisiert an
        for src, target in mapping.items():
            arr[arr == src] = target
        
        # Erstelle ein neues Bild aus dem modifizierten Array im "P"-Modus
        new_img = Image.fromarray(arr.astype(np.uint8), mode="P")
        # Übernehme die Palette, falls vorhanden
        palette = img.getpalette()
        if palette is not None:
            new_img.putpalette(palette)
        
        new_img.save(output_path)
    except Exception as e:
        print(f"Fehler bei {filename}: {e}")

def process_folder(input_folder, output_folder, mapping, num_workers=8):
    """
    Verarbeitet alle Bilder im input_folder parallel und speichert
    die relabelten Bilder in output_folder.
    """
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg'))]
    print(f"{len(files)} Dateien werden verarbeitet...")
    
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(process_image, filename, input_folder, output_folder, mapping)
                   for filename in files]
        
        for _ in tqdm(as_completed(futures), total=len(futures), desc="Processing Images"):
            pass

# Mapping for Relabeling Mapillary Vistas to big Class Dataset

In [ ]:
# Mapping-Dictionary:
# - "only"-Klassen werden auf die Hauptklasse gemappt (z. B. 55 → 35).
# - Alle in Cityscapes ausgeschlossenen Klassen werden ebenfalls entfernt (unlabeled = 123).
# - Verkehrsrelevante Klassen (Ampeln, Schilder, Markierungen) bleiben erhalten!

mapping = {
    # ✅ Fahrbahnmarkierungen (vereinheitlicht auf Hauptklassen)
    55: 35,  # Lane Marking (only) - Dashed Line → Lane Marking - Dashed Line
    56: 45,  # Lane Marking (only) - Crosswalk → Lane Marking - Crosswalk
    57: 50,  # Lane Marking (only) - Other → Lane Marking - Other
    58: 54,  # Lane Marking (only) - Test → Lane Marking - Text

    # ❌ Entfernte Klassen (bereits in Cityscapes ausgeschlossen):
    2: 123,   # Ambiguous Barrier → Unlabeled
    8: 123,   # Road Median → Unlabeled
    9: 123,   # Road Side → Unlabeled
    10: 123,  # Lane Separator → Unlabeled
    17: 123,  # Parking → Unlabeled
    18: 123,  # Parking Aisle → Unlabeled
    20: 123,  # Rail Track → Unlabeled
    23: 123,  # Service Lane → Unlabeled
    26: 123,  # Bridge → Unlabeled
    28: 123,  # Garage → Unlabeled
    29: 123,  # Tunnel → Unlabeled
    60: 123,  # Sand → Unlabeled
    115: 123, # Vehicle Group → Unlabeled
    119: 123, # Dynamic → Unlabeled
    121: 123, # Ground → Unlabeled
    122: 123, # Static → Unlabeled
}


### Für das Trainingsdatenset

In [ ]:
run = False

if run: 
    input_folder = "Mapillary_Vistas/training/v2.0/labels"         # Ordner mit den Originalbildern
    output_folder = "Mapillary_Vistas/training/v2.0/labels_big"  # Zielordner für die relabelten Bilder

    # Verarbeitung starten
    process_folder(input_folder, output_folder, mapping, num_workers=8)


### Für das Validierungsdatenset

In [ ]:
run = False

if run: 
    input_folder = "Mapillary_Vistas/validation/v2.0/labels"         # Ordner mit den Originalbildern
    output_folder = "Mapillary_Vistas/validation/v2.0/labels_big"  # Zielordner für die relabelten Bilder

    # Verarbeitung starten
    process_folder(input_folder, output_folder, mapping, num_workers=8)


# Mapping for Relabeling Mapillary Vistas to small Class Dataset

In [5]:
# Mapping-Dictionary:
# - "only"-Klassen werden auf die Hauptklasse gemappt (z. B. 55 → 35).
# - Alle in Cityscapes ausgeschlossenen Klassen werden ebenfalls entfernt (unlabeled = 123).
# - Verkehrsrelevante Klassen (Ampeln, Schilder, Markierungen) bleiben erhalten!

mapping = {
    # ✅ Fahrbahnmarkierungen (vereinheitlicht auf Hauptklassen)
    55: 35,  # Lane Marking (only) - Dashed Line → Lane Marking - Dashed Line
    56: 45,  # Lane Marking (only) - Crosswalk → Lane Marking - Crosswalk
    57: 50,  # Lane Marking (only) - Other → Lane Marking - Other
    58: 54,  # Lane Marking (only) - Test → Lane Marking - Text

    # ❌ Entfernte Klassen (bereits in Cityscapes ausgeschlossen):
    2: 123,   # Ambiguous Barrier → Unlabeled
    8: 123,   # Road Median → Unlabeled
    9: 123,   # Road Side → Unlabeled
    10: 123,  # Lane Separator → Unlabeled
    17: 123,  # Parking → Unlabeled
    18: 123,  # Parking Aisle → Unlabeled
    20: 123,  # Rail Track → Unlabeled
    23: 123,  # Service Lane → Unlabeled
    26: 123,  # Bridge → Unlabeled
    28: 123,  # Garage → Unlabeled
    29: 123,  # Tunnel → Unlabeled
    60: 123,  # Sand → Unlabeled
    115: 123, # Vehicle Group → Unlabeled
    119: 123, # Dynamic → Unlabeled
    121: 123, # Ground → Unlabeled
    122: 123, # Static → Unlabeled

    # ❌ Weitere irrelevante Klassen entfernt (NICHT in Cityscapes vorhanden):
    0: 123,   # Bird → Unlabeled
    1: 123,   # Ground Animal → Unlabeled
    59: 123,  # Mountain → Unlabeled
    62: 123,  # Snow → Unlabeled
    63: 123,  # Terrain → Unlabeled
    65: 123,  # Water → Unlabeled
    66: 123,  # Banner → Unlabeled
    67: 123,  # Bench → Unlabeled
    68: 123,  # Bike Rack → Unlabeled
    69: 123,  # Catch Basin → Unlabeled
    70: 123,  # CCTV Camera → Unlabeled
    71: 123,  # Fire Hydrant → Unlabeled
    72: 123,  # Junction Box → Unlabeled
    73: 123,  # Mailbox → Unlabeled
    74: 123,  # Manhole → Unlabeled
    75: 123,  # Parking Meter → Unlabeled
    76: 123,  # Phone Booth → Unlabeled
    77: 123,  # Pothole → Unlabeled
    78: 123,  # Signage - Advertisement → Unlabeled
    79: 123,  # Signage - Ambiguous → Unlabeled
    80: 123,  # Signage - Back → Unlabeled
    81: 123,  # Signage - Information → Unlabeled
    82: 123,  # Signage - Other → Unlabeled
    83: 123,  # Signage - Store → Unlabeled
    87: 123,  # Traffic Sign Frame → Unlabeled
    88: 123,  # Utility Pole → Unlabeled
    89: 123,  # Traffic Cone → Unlabeled
    96: 123,  # Traffic Sign - Ambiguous → Unlabeled
    97: 123,  # Traffic Sign (Back) → Unlabeled
    98: 123,  # Traffic Sign - Direction (Back) → Unlabeled
    101: 123, # Traffic Sign - Parking → Unlabeled
    104: 123, # Trash Can → Unlabeled
    106: 123, # Boat → Unlabeled
    109: 123, # Caravan → Unlabeled
    113: 123, # Trailer → Unlabeled
    117: 123, # Water Valve → Unlabeled
    118: 123, # Car Mount → Unlabeled
    120: 123  # Ego Vehicle → Unlabeled
}


### Für das Trainingsdatenset

In [ ]:
run = False

if run: 
    input_folder = "Mapillary_Vistas/training/v2.0/labels"         # Ordner mit den Originalbildern
    output_folder = "Mapillary_Vistas/training/v2.0/labels_small"  # Zielordner für die relabelten Bilder

    # Verarbeitung starten
    process_folder(input_folder, output_folder, mapping, num_workers=8)

### Für das Validierungsdatenset

In [ ]:
run = False

if run: 
    input_folder = "Mapillary_Vistas/validation/v2.0/labels"         # Ordner mit den Originalbildern
    output_folder = "Mapillary_Vistas/validation/v2.0/labels_small"  # Zielordner für die relabelten Bilder

    # Verarbeitung starten
    process_folder(input_folder, output_folder, mapping, num_workers=8)


2000 Dateien werden verarbeitet...


Processing Images:   0%|          | 0/2000 [00:00<?, ?it/s]